<a href="https://colab.research.google.com/github/elias-mena/Uber-Data/blob/main/eats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


Uber give us a data set with the order info and other with the restaurant info

In [8]:
%cd '/content/drive/My Drive/Colab Notebooks/Uber'
!ls

/content/drive/My Drive/Colab Notebooks/Uber
eats.ipynb		eats_restaurant_names.csv  trips.ipynb
eats_order_details.csv	trips_data.csv


In [109]:
import pandas as pd 
import numpy as np
import plotly.express as px # Interactive plots
from datetime import datetime

## Exploring the data sets

Original data frame from Uber

In [61]:
eats_df = pd.read_csv('eats_order_details.csv')
eats_df.shape

(7, 11)

In [14]:
eats_df.sample(3)

,Territory,Restaurant ID,Order ID,Order Time,Order Status,Item Name,Customizations,Special Instructions,Item Price,Order Price,Currency
0,od,5d7e3011-247f-4ca3-810c-e34a622ecafa,a5795aca-eae7-4b18-9bc8-e07368839d47,2021-08-05 05:20:03 +0000 UTC,completed,Papas fritas pequeñas,NaN,NaN,1800,5455,CRC
5,od,344619a8-98fa-437e-84a8-3f343e60bc1f,04f14bcd-e93c-43e3-91b0-f32bff8dc5e1,2018-09-25 20:26:44 +0000 UTC,completed,Chifrijo,NaN,NaN,2500,4350,CRC
4,od,055f6efc-8c25-4adf-89f2-bf621e884236,9004bb17-b782-431b-9dc6-aac837d2a6c3,2021-06-10 18:29:42 +0000 UTC,canceled,Mouse inalámbrico verde XTM-310 Xtech 400 grams,NaN,NaN,4990,0,CRC


Variables info

In [11]:
eats_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Territory             7 non-null      object 
 1   Restaurant ID         7 non-null      object 
 2   Order ID              7 non-null      object 
 3   Order Time            7 non-null      object 
 4   Order Status          7 non-null      object 
 5   Item Name             7 non-null      object 
 6   Customizations        2 non-null      object 
 7   Special Instructions  0 non-null      float64
 8   Item Price            7 non-null      int64  
 9   Order Price           7 non-null      int64  
 10  Currency              7 non-null      object 
dtypes: float64(1), int64(2), object(8)
memory usage: 744.0+ bytes


In [62]:
rest_df = pd.read_csv('eats_restaurant_names.csv')
rest_df.shape

(5, 4)

As we can see, our data frames have a common column, so we can join them

In [17]:
rest_df.sample(3)

,City,Restaurant ID,Restaurant Name,Order Time
0,"San Jose, CR",5d7e3011-247f-4ca3-810c-e34a622ecafa,Comida China Liu Liu,2021-08-05 04:52:43
4,"San Jose, CR",344619a8-98fa-437e-84a8-3f343e60bc1f,Pollos El Pueblo,2018-09-25 19:38:42
3,"San Jose, CR",055f6efc-8c25-4adf-89f2-bf621e884236,Universal - Multiplaza Curridabat 🛒💐,2021-06-10 18:29:11


We need only the id and the name of the restaurant from this data frame, so lest take only those columns

In [63]:
rest_df = rest_df[['Restaurant ID','Restaurant Name']]
rest_df.head(6)

,Restaurant ID,Restaurant Name
0,5d7e3011-247f-4ca3-810c-e34a622ecafa,Comida China Liu Liu
1,36debefc-7885-4274-a458-1c2789593064,Wing Republic - Guadalupe
2,05292868-06f5-473f-bbb3-0e492bea2e3e,RadioShack - Novacentro 🛒💐
3,055f6efc-8c25-4adf-89f2-bf621e884236,Universal - Multiplaza Curridabat 🛒💐
4,344619a8-98fa-437e-84a8-3f343e60bc1f,Pollos El Pueblo


And we are gonna take only the columns we need from the orders data frame too

In [64]:
eats_df = eats_df[['Restaurant ID','Order Time','Order Status','Item Name','Item Price','Order Price']]
eats_df.head(6)

,Restaurant ID,Order Time,Order Status,Item Name,Item Price,Order Price
0,5d7e3011-247f-4ca3-810c-e34a622ecafa,2021-08-05 05:20:03 +0000 UTC,completed,Papas fritas pequeñas,1800,5455
1,5d7e3011-247f-4ca3-810c-e34a622ecafa,2021-08-05 05:20:03 +0000 UTC,completed,Arroz cantones entero,2700,5455
2,36debefc-7885-4274-a458-1c2789593064,2021-07-21 02:24:20 +0000 UTC,completed,6 Alitas,4500,3235
3,05292868-06f5-473f-bbb3-0e492bea2e3e,2021-06-10 19:04:15 +0000 UTC,completed,2604700 Radioshack Mouse Óptico Usb / 2604700 ...,2850,4307
4,055f6efc-8c25-4adf-89f2-bf621e884236,2021-06-10 18:29:42 +0000 UTC,canceled,Mouse inalámbrico verde XTM-310 Xtech 400 grams,4990,0
5,344619a8-98fa-437e-84a8-3f343e60bc1f,2018-09-25 20:26:44 +0000 UTC,completed,Chifrijo,2500,4350


Now we can merge the data frames to get the Restaurant Name column

In [78]:
new_df = pd.merge(rest_df, eats_df, how = 'left', on = 'Restaurant ID')
new_df.drop('Restaurant ID', inplace=True, axis=1) # Now we dont need the id
new_df.sample(3)

,Restaurant Name,Order Time,Order Status,Item Name,Item Price,Order Price
5,Pollos El Pueblo,2018-09-25 20:26:44 +0000 UTC,completed,Chifrijo,2500,4350
2,Wing Republic - Guadalupe,2021-07-21 02:24:20 +0000 UTC,completed,6 Alitas,4500,3235
6,Pollos El Pueblo,2018-09-25 20:26:44 +0000 UTC,completed,Nuggets,1850,4350


## Cleaning the data

We will choose only the completed orders

In [79]:
new_df = new_df[new_df['Order Status'] == "completed"]
new_df = new_df.drop('Order Status', axis=1) # Now we dont need the status
new_df.sample(3)

,Restaurant Name,Order Time,Item Name,Item Price,Order Price
2,Wing Republic - Guadalupe,2021-07-21 02:24:20 +0000 UTC,6 Alitas,4500,3235
5,Pollos El Pueblo,2018-09-25 20:26:44 +0000 UTC,Chifrijo,2500,4350
0,Comida China Liu Liu,2021-08-05 05:20:03 +0000 UTC,Papas fritas pequeñas,1800,5455


Change the name of the colums

In [80]:
new_df.rename(columns={'Restaurant Name':'Restaurant','Order Time':'Time','Item Name':'Item','Order Price':'Amount'}, inplace=True)
new_df.head(2)

,Restaurant,Time,Item,Item Price,Amount
0,Comida China Liu Liu,2021-08-05 05:20:03 +0000 UTC,Papas fritas pequeñas,1800,5455
1,Comida China Liu Liu,2021-08-05 05:20:03 +0000 UTC,Arroz cantones entero,2700,5455


Look at the variable types, we need to change it to can manage the data properly

In [81]:
new_df.dtypes

Restaurant    object
Time          object
Item          object
Item Price     int64
Amount         int64
dtype: object

In [82]:
# Categorical variables
new_df[['Restaurant','Item']] = new_df[['Restaurant','Item']].astype('category')

# Numeric variables
new_df[['Amount']] = new_df[['Amount']].astype('int')
new_df[['Item Price']] = new_df[['Item Price']].astype('int')

# Date variables
new_df['Time'] = new_df['Time'].str[:10]
new_df['Time'] = pd.to_datetime(new_df['Time'])

# A new data frame with the clean data
eats_data = new_df

# Now look at the types in the new data frame
eats_data.dtypes

Restaurant          category
Time          datetime64[ns]
Item                category
Item Price             int64
Amount                 int64
dtype: object

We need some others colums with the day, month and week

In [83]:
eats_data['Day'] = eats_data['Time'].dt.strftime("%A")
eats_data['Month'] = eats_data['Time'].dt.strftime("%B")
eats_data['Week'] = eats_data['Time'].dt.strftime("%W")

eats_data.head(3)

,Restaurant,Time,Item,Item Price,Amount,Day,Month,Week
0,Comida China Liu Liu,2021-08-05,Papas fritas pequeñas,1800,5455,Thursday,August,31
1,Comida China Liu Liu,2021-08-05,Arroz cantones entero,2700,5455,Thursday,August,31
2,Wing Republic - Guadalupe,2021-07-21,6 Alitas,4500,3235,Wednesday,July,29


## Plotting the data

### Favorite Restaurant

In [87]:
df_product = eats_data.groupby("Restaurant")["Restaurant"].count().to_frame()
df_product.rename(columns={'Restaurant':'Count'}, inplace=True)
df_product = df_product.reset_index()
df_product

,Restaurant,Count
0,Comida China Liu Liu,2
1,Pollos El Pueblo,2
2,RadioShack - Novacentro 🛒💐,1
3,Wing Republic - Guadalupe,1


In [89]:
px.bar(df_product, x = "Restaurant", y = "Count",  title = "Favorite Restaurant")

### Monthly Orders

In [99]:
df_monthly_orders = eats_data.groupby("Month")["Month"].count().to_frame()
df_monthly_orders.rename(columns={'Month':'Orders'}, inplace=True)
df_monthly_orders = df_monthly_orders.reset_index()
df_monthly_orders.head(4)

,Month,Orders
0,August,2
1,July,1
2,June,1
3,September,2


Statictics about the monthly trips

In [100]:
df_monthly_orders['Orders'].describe()

count    4.00000
mean     1.50000
std      0.57735
min      1.00000
25%      1.00000
50%      1.50000
75%      2.00000
max      2.00000
Name: Orders, dtype: float64

Orders by month

In [101]:
px.bar_polar(df_monthly_orders, r="Orders", theta="Month",   title = "Orders by month")

### Daily Trips

In [102]:
df_daily_trips = eats_data.groupby("Day")["Day"].count().to_frame()
df_daily_trips.rename(columns={'Day':'Orders'}, inplace=True)
df_daily_trips = df_daily_trips.reset_index()
df_daily_trips

,Day,Orders
0,Thursday,3
1,Tuesday,2
2,Wednesday,1


Statictics about the monthly trips

In [103]:
df_daily_trips["Orders"].describe()

count    3.0
mean     2.0
std      1.0
min      1.0
25%      1.5
50%      2.0
75%      2.5
max      3.0
Name: Orders, dtype: float64

In [104]:
px.bar_polar(df_daily_trips, r="Orders", theta="Day",   title = "Orders by day")

### Monthly Spences

In [90]:
df_monthly_spences = eats_data.groupby("Month")["Amount"].sum().to_frame()
df_monthly_spences = df_monthly_spences.reset_index()
df_monthly_spences.head(4)

,Month,Amount
0,August,10910
1,July,3235
2,June,4307
3,September,8700


Statistics about monthly spences

In [91]:
df_monthly_spences["Amount"].describe()

count        4.000000
mean      6788.000000
std       3625.180363
min       3235.000000
25%       4039.000000
50%       6503.500000
75%       9252.500000
max      10910.000000
Name: Amount, dtype: float64

Trend on monthly spences

In [92]:
px.line(df_monthly_spences, x = "Month", y = "Amount",  title = "Monthly Spences")

### Weekly Spences

In [105]:
df_weekly_spences = eats_data.groupby("Week")["Amount"].sum().to_frame()
df_weekly_spences = df_weekly_spences.reset_index()
df_weekly_spences.head(4)

,Week,Amount
0,23,4307
1,29,3235
2,31,10910
3,39,8700


Statistics about weekly spences

In [106]:
df_weekly_spences['Amount'].describe()

count        4.000000
mean      6788.000000
std       3625.180363
min       3235.000000
25%       4039.000000
50%       6503.500000
75%       9252.500000
max      10910.000000
Name: Amount, dtype: float64

Trend on weekly spences

In [107]:
px.line(df_weekly_spences, x = "Week", y = "Amount",  title = "Weekly Spences")

### Fare Amount histogram

In [108]:
px.histogram(eats_data, x = "Amount",  title = "Fare Amount")